In [13]:
import pandas as pd
import numpy as np

In [14]:
DIR = "/Users/damoncrockett/astrodata/tables/"

In [15]:
d = pd.read_csv(DIR+"astrodata_design_matrix.csv")

In [16]:
df = pd.read_csv(DIR+"astrodata_moon_galaxy.csv")

In [17]:
d = d[df.galaxy==1]

In [18]:
from sklearn.manifold import TSNE as tsne
from PIL import Image, ImageDraw

In [19]:
thumb_side = 128 # publication size

## Fill

In [20]:
from shapely.geometry import Point

In [21]:
crit = np.sqrt(2) # adjacency

In [22]:
num_bin = 3 * int(round( np.sqrt( len(d) / np.pi ) )) # circle that fits square, fill 'factor'==3

In [23]:
X = d.iloc[:,1:].as_matrix()

In [24]:
tsne_obj = tsne(n_components=2)

subspace_tsne = pd.DataFrame(tsne_obj.fit_transform(X),columns=["x","y"])
subspace_tsne['grid_x'] = pd.cut(subspace_tsne['x'],num_bin,labels=False)
subspace_tsne['grid_y'] = pd.cut(subspace_tsne['y'],num_bin,labels=False)
subspace_tsne['local_path'] = list(d.local_path)

In [25]:
grid_side = num_bin 
px_w = thumb_side * grid_side
px_h = thumb_side * grid_side

x,y = range(grid_side) * grid_side, np.repeat(range(grid_side),grid_side)
grid_list = pd.DataFrame(x,columns=['x'])
grid_list['y'] = y

point = []
for i in range(len(grid_list)):
    point.append(Point(grid_list.x.loc[i],grid_list.y.loc[i]))

grid_list['point'] = point

open_grid = list(grid_list.point)
canvas = Image.new('RGB',(px_w,px_h),(0,0,0))
plotted_dict = {}

plotted = 0
for i in np.random.permutation(np.arange(np.round(len(subspace_tsne)))):
    im = Image.open(subspace_tsne.local_path.loc[i])
    im.thumbnail((thumb_side,thumb_side),Image.ANTIALIAS)
    x = subspace_tsne.grid_x.loc[i] * thumb_side 
    y = subspace_tsne.grid_y.loc[i] * thumb_side 
    grid_point = Point((subspace_tsne.grid_x.loc[i],subspace_tsne.grid_y.loc[i]))

    if grid_point in open_grid:
        canvas.paste(im,(x,y))
        open_grid.remove(grid_point)
        plotted_dict[grid_point] = subspace_tsne.local_path.loc[i]
        plotted+=1

    elif len(open_grid) == 0:
        break

    else:
        closest_open = min(open_grid,key=lambda x: grid_point.distance(x))
        if grid_point.distance(closest_open) <= crit: # adjacency criterion
            x = int(closest_open.x) * thumb_side
            y = int(closest_open.y) * thumb_side
            canvas.paste(im,(x,y))
            open_grid.remove(closest_open)
            plotted_dict[closest_open] = subspace_tsne.local_path.loc[i]
            plotted+=1

In [26]:
PRE = "/Users/damoncrockett/Desktop/"
prop = str(int(round(plotted*100/len(subspace_tsne))))
canvas.save(PRE+"galaxy_"+str(num_bin)+"_"+prop+"_"+"_"+str(plotted)+".png")